In [ ]:
import skimage.io as io
import os
import pandas as pd
import numpy as np

root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
data_path = os.path.join(root, "results", "20240303", "")
hf_pca_df = pd.read_csv(os.path.join(data_path, "hf_morph_df.csv"))

fig_path = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/slides/morphseq/20250312/"

image_path = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/training_data/20241107_ds/images/0/"
hf_snip_vec = hf_pca_df["snip_id"].to_numpy()
hf_time_vec = hf_pca_df["timepoint"].to_numpy()
hf_temp_vec = hf_pca_df["temperature"].to_numpy()
image_list = []
for snip_id in hf_snip_vec:
    im = io.imread(os.path.join(image_path, snip_id + ".jpg"))
    image_list.append(im)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm

image_path = os.path.join(fig_path, "cohort_images", "")
os.makedirs(image_path, exist_ok=True)
im_shape = image_list[0].shape 

for time in np.unique(hf_time_vec):
    for temp in np.unique(hf_temp_vec):
        obs_indices = np.where((hf_time_vec==time) & (hf_temp_vec==temp))[0]
        
        # fig = go.Figure() # make_subplots(rows=2, cols=4)
        
        # Add each image to a subplot
        top_list = []
        bottom_list = []
        snip_array = [[""]*4, [""]*4]
        for i in range(8):
            if len(obs_indices) > i:
                im = image_list[obs_indices[i]]
                snip = hf_snip_vec[obs_indices[i]]
            else:
                im = np.zeros(im_shape, dtype=np.uint8)
                snip = ""
                
            if i < 4:
                top_list.append(im)
                snip_array[0][i] = snip
            else:
                bottom_list.append(im)
                snip_array[1][i-4] = snip

        tiled_image = np.block([top_list,
                                bottom_list])
        
        fig = px.imshow(tiled_image, color_continuous_scale="gray", title=f"{temp:02}C @{time:02}hpf")

        # Compute tile dimensions and grid layout information.
        tile_height, tile_width = im_shape
        n_cols = 4
        n_rows = 2
        # Margin (in pixel units) for positioning the text above the bottom edge.
        margin = 10

        # Create a list of annotations – one per tile.
        annotations = []
        for r in range(n_rows):
            for c in range(n_cols):
                # Calculate the center x-coordinate for the current tile.
                center_x = c * tile_width + tile_width / 2
                # Calculate the y-coordinate near the bottom of the tile.
                # (y coordinates increase downward in images.)
                y_bottom = (r + 1) * tile_height - margin
                # Create a label; here we use a placeholder. You can change this to your desired text.
                # label_text = f"Tile {r * n_cols + c + 1}"
                annotations.append(dict(
                    x=center_x,
                    y=y_bottom,
                    xref="x",
                    yref="y",
                    text=snip_array[r][c],
                    showarrow=False,
                    font=dict(color="white", size=12),
                    xanchor="center",
                    yanchor="bottom"
                ))

        # Update the layout to include the annotations.
        fig.update_layout(annotations=annotations)
        
        # Update layout for better display
        fig.update_layout(
            height=1200,
            width=2400
        )
        
        fig.write_image(image_path + f"embryo_images_tp{time:02}_temp{temp:02}.png", engine="kaleido")


In [ ]:
snip